## Pair Programming - Regresión logística 5

### Decision Tree

---

In [ ]:
#librerías

**Objetivos**:
### 1. Ajustad un modelo de Decision Tree a nuestros datos.

### 2. Calculad las métricas a nuestro nuevo modelo.

### 3. Comparad las métricas con el modelo hecho hasta ahora. ¿Cuál es mejor?